In [ ]:
# Step 1 - 3

In [ ]:
''' 
1) Split the dataset into D1 and D2. Make sure there is similar proportion of treated (e401=1) in both groups.
2) Start with D1. Split D1 into two:
    a) D1_1 with e401 = 1
    b) D1_0 with e401 = 0
3) Train two models, one for D1_1, and another for D1_0, f1 and f0, respectively (as both models use 
Random Forest or Boosting)
'''

In [61]:
import numpy as np
import pandas as pd
import doubleml as dml
from doubleml.datasets import fetch_401K
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [54]:
#code
data = fetch_401K(return_type='DataFrame')
data

,nifa,net_tfa,tw,age,inc,fsize,educ,db,marr,twoearn,e401,p401,pira,hown
0,0.0,0.0,4500.0,47,6765.0,2,8,0,0,0,0,0,0,1
1,6215.0,1015.0,22390.0,36,28452.0,1,16,0,0,0,0,0,0,1
2,0.0,-2000.0,-2000.0,37,3300.0,6,12,1,0,0,0,0,0,0
3,15000.0,15000.0,155000.0,58,52590.0,2,16,0,1,1,0,0,0,1
4,0.0,0.0,58000.0,32,21804.0,1,11,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9910,98498.0,98858.0,157858.0,52,73920.0,1,16,1,0,0,1,1,0,1
9911,287.0,6230.0,15730.0,41,42927.0,4,14,0,1,1,1,1,1,1
9912,99.0,6099.0,7406.0,40,23619.0,2,16,1,0,0,1,0,1,0
9913,0.0,-32.0,2468.0,47,14280.0,4,6,1,0,0,1,1,0,0


In [55]:
# Split the dataset into D1 and D2. Make sure there is similar proportion of treated (e401=1) in both groups.
D1 = data.groupby('e401', group_keys=False).apply(lambda x: x.sample(frac=0.5))
D2 = data.copy().drop(D1.index, axis=0)
D1.reset_index(inplace = True)
D2.reset_index(inplace = True)

In [56]:
#2) Start with D1. Split D1 into two: D1_1 with e401 = 1, D1_0 with e401 = 0
D1_0 = D1[D1['e401'] == 0]
D1_1 = D1[D1['e401'] == 1]

In [65]:
#3) Train two models, one for D1_1, and another for D1_0, f1 and f0, respectively 

## Data Prep

## make x and y
D1_0_X = D1_0.loc[:, D1_0.columns != 'net_tfa']
D1_1_X = D1_1.loc[:, D1_1.columns != 'net_tfa']

D1_0_y = D1_0['net_tfa']
D1_1_y = D1_1['net_tfa']

## split into train and test sets

D1_0_X_train, D1_0_X_test, D1_0_y_train, D1_0_y_test = train_test_split(D1_0_X, D1_0_y)

D1_1_X_train, D1_1_X_test, D1_1_y_train, D1_1_y_test = train_test_split(D1_1_X, D1_1_y)


## scale data

scaler_D1_0 = StandardScaler()
scaler_D1_0.fit(D1_0_X_train)
D1_0_X_train_scl = scaler_D1_0.transform(D1_0_X_train)
D1_0_X_test_scl = scaler_D1_0.transform(D1_0_X_test)

scaler_D1_1 = StandardScaler()
scaler_D1_1.fit(D1_0_X_train)
D1_1_X_train_scl = scaler_D1_1.transform(D1_1_X_train)
D1_1_X_test_scl = scaler_D1_1.transform(D1_1_X_test)


RF_param_grid = {
    'n_estimators':[50, 60, 70, 80, 90, 100, 110, 120, 130],
    'min_samples_leaf':[5, 7, 8, 9, 10, 11, 12, 13, 14, 15],
}

#f0
RF_f0 = RandomForestRegressor(
    n_estimators=500, max_depth=7, max_features=3, min_samples_leaf=3)

RF_f0_gmodel = GridSearchCV(RF_f0, param_grid=RF_param_grid, scoring='r2', cv=5)


RF_f0_gmodel.fit(D1_0_X_train_scl, D1_0_y_train)


#f1
RF_f1 = RandomForestRegressor(
    n_estimators=500, max_depth=7, max_features=3, min_samples_leaf=3)

RF_f1_gmodel = GridSearchCV(RF_f1, param_grid=RF_param_grid, scoring='r2', cv=5)

RF_f1_gmodel.fit(D1_1_X_train_scl, D1_1_y_train)

/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3

/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3

/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3

/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3

/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3

...

/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3

/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3

/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3

/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLDataProcessing/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_depth=7, max_features=3,
                                             min_samples_leaf=3,
                                             n_estimators=500),
             param_grid={'max_features': ['log2', 'auto', 'sqrt'],
                         'min_samples_leaf': [5, 7, 8, 9, 10, 11, 12, 13, 14,
                                              15],
                         'n_estimators': [50, 60, 70, 80, 90, 100, 110, 120,
                                          130]},
             scoring='r2')

In [64]:
# Step 4-6

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [ ]:
'''
4) Make predictions on the complementary dataset D2, and calculate the ATE_2 as the average of f1(D2) - f0(D2)
5) Repeat the process switching roles between D1 and D2, and calculate the ATE = (ATE_1 + ATE_2)/2
6) Include the previous process inside a ‘for’ loop.
'''

In [ ]:
#code

In [ ]:
#Step 7 & 8 and documentation

In [ ]:
'''
7) Pick a random sample (with replacement) of the original dataset D, which we will call D_boostrap, and calculate the ATE with the process described in the previous slide of the D_boostrap
8) Get the results for all the resamplings and calculate the mean and confidence intervals (quantiles 2.5%, 97.5%) of the ATEs.
'''

In [ ]:
#code